In [1]:
import xml.dom.minidom
import xml.etree.ElementTree as ET
from xml.dom.minidom import Node
import pandas as pd

In [2]:
# HAI Summary Files
CLABSI_Summary = pd.read_csv("CLABSI Summary.csv")
CAUTI_Summary = pd.read_csv("CAUTI Summary.csv")
VAE_Summary = pd.read_csv("VAE Summary.csv")
MDRO_Summary = pd.read_csv("MDRO Summary.csv")

In [3]:
# HAI Individual Patient Files
CLABSI_Individual = pd.read_csv("CLABSI Individuals.csv")
CAUTI_Individual = pd.read_csv("CAUTI Individuals.csv")
VAE_Individual = pd.read_csv("VAE Individuals.csv")
MDRO_Individual = pd.read_csv("MDRO Individuals.csv")

In [4]:
# convert the 'SpecimenDate', 'FacAdmissionDate', and 'UnitAdmissionDate' columns to datetime format
MDRO_Individual.SpecimenDate = pd.to_datetime(MDRO_Individual.SpecimenDate)
MDRO_Individual.FacAdmissionDate = pd.to_datetime(MDRO_Individual.FacAdmissionDate)
MDRO_Individual.UnitAdmissionDate = pd.to_datetime(MDRO_Individual.UnitAdmissionDate)
# Change the date formating
MDRO_Individual.SpecimenDate = MDRO_Individual.SpecimenDate.apply(lambda x: x.strftime('%m/%d/%Y'))
MDRO_Individual.FacAdmissionDate = MDRO_Individual.FacAdmissionDate.apply(lambda x: x.strftime('%m/%d/%Y'))
MDRO_Individual.UnitAdmissionDate = MDRO_Individual.UnitAdmissionDate.apply(lambda x: x.strftime('%m/%d/%Y'))

## HAI Summary Function

In [5]:
# The variables needed for this function are the: Hospital abbreviation, Hospital NDNQI template, Hospital NDNQI
# code, Calendar year being submitted, and Calendar quarter being submitted
def NDNQI_HAI_Summary_XML(Selected_Hosp,Hosp_XML_Template,Hosp_NDNQI_Code,Calendar_Year,Calendar_Quarter):
    # This section is to add or create each HAI summary section
    
    # This Hospital's Summary Data
    Hosp_MDRO_data = MDRO_Summary[MDRO_Summary.Hospital == Selected_Hosp]
    Hosp_CLABSI_data = CLABSI_Summary[CLABSI_Summary.Hospital == Selected_Hosp]
    Hosp_CAUTI_data = CAUTI_Summary[CAUTI_Summary.Hospital == Selected_Hosp]
    Hosp_VAE_data = VAE_Summary[VAE_Summary.Hospital == Selected_Hosp]
    # A List of this Hospital's Unit Names
    Hosp_Units = list(set(Hosp_MDRO_data.UnitName))

    #This is the current XML template for this hospital and their NDNQI units for this metric
    tree = xml.dom.minidom.parse(Hosp_XML_Template)
    # This is each unit node of the xml file
    unit_nodes = tree.getElementsByTagName("UnitYrQtr")

    # Make sure the correct year and quarter are listed
    H = tree.getElementsByTagName("HospitalCode")[0].childNodes[0].nodeValue = Hosp_NDNQI_Code
    Y = tree.getElementsByTagName("Year")[0].childNodes[0].nodeValue = Calendar_Year
    Q = tree.getElementsByTagName("Quarter")[0].childNodes[0].nodeValue = Calendar_Quarter


    # Take the xml UnitName and replace it with the correct Hospital UnitName from the Hosp_Units list. 
    for idx, unit in enumerate(unit_nodes):
        unit.getElementsByTagName('UnitName')[0].childNodes[0].nodeValue = Hosp_Units[idx]
        current_xml_unit = unit.getElementsByTagName('UnitName')[0].childNodes[0].nodeValue

        # Now grab each of the 3 months for that unit from the Hospital data
        MDRO_new_data = Hosp_MDRO_data[Hosp_MDRO_data.UnitName == Hosp_Units[idx]]
        CLABSI_new_data = Hosp_CLABSI_data[Hosp_CLABSI_data.UnitName == Hosp_Units[idx]]
        CAUTI_new_data = Hosp_CAUTI_data[Hosp_CAUTI_data.UnitName == Hosp_Units[idx]]
        VAE_new_data = Hosp_VAE_data[Hosp_VAE_data.UnitName == Hosp_Units[idx]]
        # Sort the data by month
        MDRO_new_data = MDRO_new_data.sort_values('Month')
        CLABSI_new_data = CLABSI_new_data.sort_values('Month')
        CAUTI_new_data = CAUTI_new_data.sort_values('Month')
        VAE_new_data = VAE_new_data.sort_values('Month')


        # Take each xml Month and replace it with each listed month from the new_data for that unit
        month_nodes = unit.getElementsByTagName("UnitMonth")
        for idx2, month in enumerate(month_nodes):
            month.getElementsByTagName('Month')[0].childNodes[0].nodeValue = MDRO_new_data.Month.iloc[idx2]

            # Take each xml MDROSummary
            MDRO_nodes = month.getElementsByTagName("MDROSummary")
            # Now replace each xml metric value with data that has the same name from our data source
            for MDRO_node in MDRO_nodes:

                for column in MDRO_new_data.columns[3:]:
                    new_value = MDRO_new_data[column].iloc[idx2]
                    MDRO_node.getElementsByTagName(column)[0].childNodes[0].nodeValue = new_value


            # Next check if the current unit is a CLABSI unit
            if current_xml_unit in list(set(CLABSI_new_data.UnitName)):

                # Create a new 'BsiSummary' to append to the current 'BsiSummarys'
                new_CLABSI_summary = tree.createElement("BsiSummary")

                CLABSI_list = ["TotalBsiDays", 'BsiDaysForBirthWeightLt751', 'BsiDaysForBirthWeight751to1000',
                              'BsiDaysForBirthWeight1001to1500', 'BsiDaysForBirthWeight1501to2500',
                              'BsiDaysForBirthWeightGt2500', 'TotalBsi', 'BsiForBirthWeightLt751',
                              'BsiForBirthWeight751to1000', 'BsiForBirthWeight1001to1500',
                              'BsiForBirthWeight1501to2500', 'BsiForBirthWeightGt2500', 'BsiPatients']

                for data in CLABSI_list:
                    # I create an element for each individual's metrics
                    data_name = tree.createElement(data)

                    if data in ['BsiDaysForBirthWeightLt751', 'BsiDaysForBirthWeight751to1000',
                              'BsiDaysForBirthWeight1001to1500', 'BsiDaysForBirthWeight1501to2500',
                              'BsiDaysForBirthWeightGt2500','BsiForBirthWeightLt751',
                              'BsiForBirthWeight751to1000', 'BsiForBirthWeight1001to1500',
                              'BsiForBirthWeight1501to2500', 'BsiForBirthWeightGt2500']:
                        data_name.setAttribute('xsi:nil', 'true')
                    elif data == 'BsiPatients':
                        data_name.appendChild(tree.createTextNode(''))
                    else:
                        # and now add the correct value for that metric
                        data_name.appendChild(tree.createTextNode(str(CLABSI_new_data[data].iloc[idx2])))

                    new_CLABSI_summary.appendChild(data_name)

                # Finally, I can add this new 'BsiSummary' to the current 'BsiSummarys'
                month.getElementsByTagName("BsiSummarys")[0].appendChild(new_CLABSI_summary)


            # Next check if the current unit is a CAUTI unit
            if current_xml_unit in list(set(CAUTI_new_data.UnitName)):

                # Create a new 'UtiSummary' to append to the current 'UtiSummarys'
                new_CAUTI_summary = tree.createElement("UtiSummary")

                CAUTI_list = ["TotalUtiDays", 'TotalUti','UtiPatients']

                for data in CAUTI_list:
                    # I create an element for each individual's metrics
                    data_name = tree.createElement(data)

                    if data == 'UtiPatients':
                        data_name.appendChild(tree.createTextNode(''))
                    else:
                        # and now add the correct value for that metric
                        data_name.appendChild(tree.createTextNode(str(CAUTI_new_data[data].iloc[idx2])))

                    new_CAUTI_summary.appendChild(data_name)

                 # Finally, I can add this new 'UtiSummary' to the current 'UtiSummarys'
                month.getElementsByTagName("UtiSummarys")[0].appendChild(new_CAUTI_summary)

            # Next check if the current unit is a VAE unit
            if current_xml_unit in list(set(VAE_new_data.UnitName)):

                # Create a new 'VAESummary' to append to the current 'VAESummarys'
                new_VAE_summary = tree.createElement("VAESummary")

                VAE_list = ["TotalVentDays",'TotalVAE','APRVReporting','TotalAPRVDays','VAEPatients']

                for data in VAE_list:
                    # I create an element for each individual's metrics
                    data_name = tree.createElement(data)

                    if data == 'TotalAPRVDays':
                        data_name.setAttribute('xsi:nil', 'true')
                    elif data == 'VAEPatients':
                        data_name.appendChild(tree.createTextNode(''))
                    else:
                        # and now add the correct value for that metric
                        data_name.appendChild(tree.createTextNode(str(VAE_new_data[data].iloc[idx2])))

                    new_VAE_summary.appendChild(data_name)

                # Finally, I can add this new 'VAESummary' to the current 'VAESummarys'
                month.getElementsByTagName("VAESummarys")[0].appendChild(new_VAE_summary)


    # This is a temporary xml file that I will now use to add the individual separation data.
    tree.writexml(open('Temp_{}.xml'.format(Selected_Hosp), 'w'))

## HAI Individual Detail Function

In [6]:
# The variables needed for this function are the: Hospital abbreviation, Hospital Temp XML file,
# Calendar year being submitted, and Calendar quarter being submitted
def NDNQI_HAI_Indiv_XML(Selected_Hosp,Temp_XML,Calendar_Year,Calendar_Quarter):
    # This is the temporary xml file that has already had the HAI Summary section filled
    
    # This Hospital's Summary Data
    Hosp_MDRO_data = MDRO_Summary[MDRO_Summary.Hospital == Selected_Hosp]
    Hosp_CLABSI_data = CLABSI_Summary[CLABSI_Summary.Hospital == Selected_Hosp]
    Hosp_CAUTI_data = CAUTI_Summary[CAUTI_Summary.Hospital == Selected_Hosp]
    Hosp_VAE_data = VAE_Summary[VAE_Summary.Hospital == Selected_Hosp]
    
    # This Hospital's Individual Data
    Hosp_MDRO_individual = MDRO_Individual[MDRO_Individual.Hospital == Selected_Hosp]
    Hosp_CLABSI_individual = CLABSI_Individual[CLABSI_Individual.Hospital == Selected_Hosp]
    Hosp_CAUTI_individual = CAUTI_Individual[CAUTI_Individual.Hospital == Selected_Hosp]
    Hosp_VAE_individual = VAE_Individual[VAE_Individual.Hospital == Selected_Hosp]
    
    tree = xml.dom.minidom.parse(Temp_XML)
    unit_nodes = tree.getElementsByTagName("UnitYrQtr")

    # This loops through each listed unit in our Temp xml file
    for idx, unit in enumerate(unit_nodes):
        each_unit = unit.getElementsByTagName('UnitName')[0].childNodes[0].nodeValue

        # Then we loop through each listed month
        month_nodes = unit.getElementsByTagName("UnitMonth")
        for idx2, month in enumerate(month_nodes):
            each_month = month.getElementsByTagName('Month')[0].childNodes[0].nodeValue

            CDIFF_count = month.getElementsByTagName('TotalCDiff')[0].childNodes[0].nodeValue
            MDRA_count = month.getElementsByTagName('TotalMRSA')[0].childNodes[0].nodeValue     

            MDROs = [int(CDIFF_count), int(MDRA_count)]
            total_MDROs = sum(MDROs)

            if total_MDROs > 0:
                # Then I grab each patient with HAI data for this unit
                MDRO_patients = Hosp_MDRO_individual[Hosp_MDRO_individual.UnitName == each_unit]
                # Lastly, I select only that unit's HAIs that occured in each specific month
                Monthly_MDROs = MDRO_patients[MDRO_patients.Month == int(each_month)]

                # Now I can take each listed HAI
                for i in range(len(Monthly_MDROs)):

                    # and create a "MDROEvent" node for each individual MDRO
                    MDRO_patient = tree.createElement("MDROEvent")

                    MDRO_patient_list = ['AgeYears','xGenderFID','xMDROOrganismFID','SpecimenDate',
                                         'FacAdmissionDate','UnitAdmissionDate','NewCDIF','NewMRSA']
                    for data in MDRO_patient_list:
                        if Monthly_MDROs['xMDROOrganismFID'].iloc[i] == 'MRSA':
                            # I create an element for each individual's metrics
                            data_name = tree.createElement(data)
                            if data == 'NewCDIF':
                                data_name.setAttribute('xsi:nil', 'true')
                            else:
                                # and I add each one to that individual's "MDROEvent" node
                                data_name.appendChild(tree.createTextNode(str(Monthly_MDROs[data].iloc[i])))

                            MDRO_patient.appendChild(data_name)
                        else:
                            # I create an element for each individual's metrics
                            data_name = tree.createElement(data)
                            if data == 'NewMRSA':
                                data_name.setAttribute('xsi:nil', 'true')
                            else:
                                # and I add each one to that individual's "MDROEvent" node
                                data_name.appendChild(tree.createTextNode(str(Monthly_MDROs[data].iloc[i])))

                            MDRO_patient.appendChild(data_name)

                    # Finally, I can add these individual MDRO patients to the "MDROEvents" section.
                    month.getElementsByTagName("MDROEvents")[0].appendChild(MDRO_patient)


            # Next check if the current unit is a CLABSI unit
            if each_unit in list(set(Hosp_CLABSI_data.UnitName)):
                CLABSI_count = month.getElementsByTagName('TotalBsi')[0].childNodes[0].nodeValue
                if int(CLABSI_count) > 0:
                    CLABSI_patients = Hosp_CLABSI_individual[Hosp_CLABSI_individual.UnitName == each_unit]
                    # Lastly, I select only that unit's HAIs that occured in each specific month
                    Monthly_CLABSI = CLABSI_patients[CLABSI_patients.Month == int(each_month)]

                    # Now I can take each listed HAI
                    for i in range(len(Monthly_CLABSI)):

                        # and create a "BsiPatient" node for each individual CLABSI
                        CLABSI_patient = tree.createElement("BsiPatient")

                        CLABSI_patient_list = ['xGenderFID','AgeYears','AgeMonths','xBirthWeightCategoryFID',
                                              'xBsiInsertionSiteFID','xBsiEventTypeFID']
                        for data in CLABSI_patient_list:
                            # I create an element for each individual's metrics
                            data_name = tree.createElement(data)

                            if data in ['AgeMonths','xBirthWeightCategoryFID','xBsiInsertionSiteFID']:
                                data_name.setAttribute('xsi:nil', 'true')
                            else:
                                # and I add each one to that individual's "BsiPatient" node
                                data_name.appendChild(tree.createTextNode(str(Monthly_CLABSI[data].iloc[i])))

                            CLABSI_patient.appendChild(data_name)

                        # Finally, I can add these individual CLABSI patients to the "BsiPatients" section.
                        month.getElementsByTagName("BsiPatients")[0].appendChild(CLABSI_patient)


            # Next check if the current unit is a CAUTI unit
            if each_unit in list(set(Hosp_CAUTI_data.UnitName)):
                CAUTI_count = month.getElementsByTagName('TotalUti')[0].childNodes[0].nodeValue
                if int(CAUTI_count) > 0:
                    CAUTI_patients = Hosp_CAUTI_individual[Hosp_CAUTI_individual.UnitName == each_unit]
                    # Lastly, I select only that unit's HAIs that occured in each specific month
                    Monthly_CAUTI = CAUTI_patients[CAUTI_patients.Month == int(each_month)]

                    # Now I can take each listed HAI
                    for i in range(len(Monthly_CAUTI)):

                        # and create a "UtiPatient" node for each individual CAUTI
                        CAUTI_patient = tree.createElement("UtiPatient")

                        CAUTI_patient_list = ['xGenderFID','AgeYears','AgeMonths','xUtiEventTypeFID']
                        for data in CAUTI_patient_list:
                            # I create an element for each individual's metrics
                            data_name = tree.createElement(data)

                            if data == 'AgeMonths':
                                data_name.setAttribute('xsi:nil', 'true')
                            else:
                                # and I add each one to that individual's "UtiPatient" node
                                data_name.appendChild(tree.createTextNode(str(Monthly_CAUTI[data].iloc[i])))

                            CAUTI_patient.appendChild(data_name)

                        # Finally, I can add these individual CAUTI patients to the "UtiPatients" section.
                        month.getElementsByTagName("UtiPatients")[0].appendChild(CAUTI_patient)

            # Next check if the current unit is a VAE unit
            if each_unit in list(set(Hosp_VAE_data.UnitName)):
                VAE_count = month.getElementsByTagName('TotalVAE')[0].childNodes[0].nodeValue
                if int(VAE_count) > 0:
                    VAE_patients = Hosp_VAE_individual[Hosp_VAE_individual.UnitName == each_unit]
                    # Lastly, I select only that unit's HAIs that occured in each specific month
                    Monthly_VAE = VAE_patients[VAE_patients.Month == int(each_month)]

                    # Now I can take each listed HAI
                    for i in range(len(Monthly_VAE)):

                        # and create a "BsiPatient" node for each individual CLABSI
                        VAE_patient = tree.createElement("VAEPatient")

                        VAE_patient_list = ['xGenderFID','AgeYears','xVAEEventTypeFID','APRVRiskFactor',
                                              'PostProcedureVAE','xVentLocationFID']
                        for data in VAE_patient_list:
                            # I create an element for each individual's metrics
                            data_name = tree.createElement(data)

                            if data in ['APRVRiskFactor','PostProcedureVAE','xVentLocationFID']:
                                data_name.setAttribute('xsi:nil', 'true')
                            else:
                                # and I add each one to that individual's "BsiPatient" node
                                data_name.appendChild(tree.createTextNode(str(Monthly_VAE[data].iloc[i])))

                            VAE_patient.appendChild(data_name)

                        # Finally, I can add these individual CLABSI patients to the "BsiPatients" section.
                        month.getElementsByTagName("VAEPatients")[0].appendChild(VAE_patient)

    # Now, we need to write and save this XML file as its own NDNQI Quarter and Year and Hospital to upload
    # into NDNQI.
    tree.writexml(open('Q{}_{} {} HAI Data.xml'.format(Calendar_Quarter, Calendar_Year,Selected_Hosp), 'w'))

## EUOSH

In [7]:
NDNQI_HAI_Summary_XML('EUOSH','2020 EUOSH HAI Template.xml','SBI556',2021,2)

In [8]:
NDNQI_HAI_Indiv_XML('EUOSH','Temp_EUOSH.xml',2021,2)

## EJCH

In [9]:
NDNQI_HAI_Summary_XML('EJCH','2020 EJCH HAI Template.xml','YTD069',2021,2)

In [10]:
NDNQI_HAI_Indiv_XML('EJCH','Temp_EJCH.xml',2021,2)

## ESJH

In [11]:
NDNQI_HAI_Summary_XML('ESJH','2020 ESJH HAI Template.xml','XAR317',2021,2)

In [12]:
NDNQI_HAI_Indiv_XML('ESJH','Temp_ESJH.xml',2021,2)

## EUHM

In [13]:
NDNQI_HAI_Summary_XML('EUHM','2020 EUHM HAI Template.xml','PVF155',2021,2)

In [14]:
NDNQI_HAI_Indiv_XML('EUHM','Temp_EUHM.xml',2021,2)

## EUH

In [15]:
NDNQI_HAI_Summary_XML('EUH','2020 EUH HAI Template.xml','JXE432',2021,2)

In [16]:
NDNQI_HAI_Indiv_XML('EUH','Temp_EUH.xml',2021,2)

## EDH

In [17]:
NDNQI_HAI_Summary_XML('EDH','2021 EDH HAI Template.xml','NMW346',2021,2)

In [18]:
NDNQI_HAI_Indiv_XML('EDH','Temp_EDH.xml',2021,2)

## EHH

In [19]:
NDNQI_HAI_Summary_XML('EHH','2021 EHH HAI Template.xml','WXX116',2021,2)

In [20]:
NDNQI_HAI_Indiv_XML('EHH','Temp_EHH.xml',2021,2)

## ELTAC

In [21]:
NDNQI_HAI_Summary_XML('ELTAC','2021 ELTAC HAI Template.xml','XLS464',2021,2)

In [22]:
NDNQI_HAI_Indiv_XML('ELTAC','Temp_ELTAC.xml',2021,2)